In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

Run the following lines to be sure that we have the latest versio of neuroconv.

In [ ]:
!pip install git+https://github.com/CatalystNeuro/roiextractors@main

# NWB and Spikeinterface

## The NWB Ecosystem

[Link to slides](https://docs.google.com/presentation/d/1DwUEKQrUkTLi2hm4RwV1egWnJ3wsGUI973Zku-HA-zg/edit?usp=sharing)

## Neurconv

### Adding Spikeinterface objects to in-memory NWB file

First let's create an NWBFile. We will work in-memory. That is, we will create an NWBFile object that is not associated with a file on disk.
We will see later how to write this NWBFile to disk.

In [ ]:
from neuroconv.tools.spikeinterface import add_electrical_series
from pynwb import NWBFile
import datetime



session_start_time = datetime.datetime.now()  # The date of today
session_description = "A test NWB file with electrical series."
identifier = "A unique identifier"

nwbfile = NWBFile(session_description=session_description, session_start_time=session_start_time, identifier=identifier)
nwbfile

We will now be adding Spikeinterface objects to the NWBFile. Let's create artificial data to demostrate the process.

In [ ]:
from spikeinterface.core import generate_ground_truth_recording


recording, sorting = generate_ground_truth_recording(num_channels=8, num_units=3, durations=[60*10.0], seed=0,)

recording = recording.rename_channels(new_channel_ids=["A", "B", "C", "D", "E", "F", "G", "H"]) 
sorting = sorting.rename_units(new_unit_ids=["Unit A", "Unit B", "Unit C"])                                  

# Add new properties to the recording and sorting
recording.set_property(key="a_channel_property", values=[f"property {channel}" for channel in recording.get_channel_ids()])
recording.set_property(key="brain_area", values=[f"Area {channel}" for channel in recording.get_channel_ids()])
sorting.set_property(key="a_unit_property", values=["property Unit A", "property Unit B", "property Unit C"])


In [ ]:
recording

In [ ]:
from probeinterface.plotting import plot_probe
probe = recording.get_probe()

plot_probe(probe);

### Recording

In [ ]:
from neuroconv.tools.spikeinterface import add_recording

add_recording(recording=recording, nwbfile=nwbfile)
nwbfile

In [ ]:
nwbfile.electrodes.to_dataframe()

In [ ]:
from spikeinterface.preprocessing import bandpass_filter

filtered_recording = bandpass_filter(recording, freq_min=1, freq_max=300)

add_electrical_series(recording=filtered_recording, nwbfile=nwbfile, write_as="lfp")
nwbfile

### Sorting

In [ ]:
from neuroconv.tools.spikeinterface import add_sorting


add_sorting(sorting=sorting, nwbfile=nwbfile)
nwbfile

In [ ]:
nwbfile.units.to_dataframe()

Finally we write to the NWBFile

In [ ]:
# Save this to a file
from pathlib import Path
from pynwb import NWBHDF5IO

nwb_file_path = Path.cwd() / "from_memory_to_nwbfile.nwb"
with NWBHDF5IO(path=nwb_file_path, mode="w") as io:
    io.write(nwbfile)

### Writing recording and sorting objects directly to NWB

Another way of working is writing the files directly to disk

In [ ]:
from pynwb import NWBHDF5IO
from neuroconv.tools.spikeinterface import write_recording
from datetime import datetime



nwbfile_path = Path.cwd() / "spikeinterface_to_nwb.nwb"

metadata = {
    "NWBFile": {"session_description": "A test NWB file with electrical series.", "identifier": "A unique identifier", "session_start_time": datetime.now()},
    "Subject": {"subject_id": "spiky_mouse" , "age": "3 months", "species": "Mus musculus", "sex": "M"},
    "Ecephys": {"device": {"name": "RecordingDevice"}}
    }

write_recording(recording=recording, nwbfile_path=nwbfile_path, metadata=metadata, overwrite=True, verbose=False)

# Load the NWB file
io = NWBHDF5IO(path=nwb_file_path, mode="r")
nwbfile = io.read()
nwbfile

Now we will close the file, then we will append the sorting object to the NWBFile on disk.

In [ ]:
io.close()

In [ ]:
from pathlib import Path
from neuroconv.tools.spikeinterface import write_sorting

# We confirm that the file that we created is stil there using pathlib
assert_msg = "Something went wrong, the file does not exist, re-run the previous cells to create the file"
assert Path(nwbfile_path).exists(), assert_msg


write_sorting(sorting=sorting, nwbfile_path=nwbfile_path, verbose=False)

In [ ]:
io = NWBHDF5IO(path=nwb_file_path, mode="r")
nwbfile = io.read()
nwbfile

## Loading NWB files in Spikeinterface

We will load the file that we created from memory as it has two electrical series and a sorting object.
First, we will confirm that our file has to electrical series

In [ ]:
from spikeinterface.extractors import NwbRecordingExtractor

nwbfile_file_path = Path.cwd() / "from_memory_to_nwbfile.nwb"
available_electrical_series = NwbRecordingExtractor.fetch_available_electrical_series_paths(file_path=nwb_file_path)
available_electrical_series

In [ ]:
recording = NwbRecordingExtractor(file_path=nwb_file_path, electrical_series_path="acquisition/ElectricalSeriesRaw")
recording

In [ ]:
from probeinterface.plotting import plot_probe
probe = recording.get_probe()

plot_probe(probe);

### Streaming objects from NWB

In [ ]:
from dandi.dandiapi import DandiAPIClient
from spikeinterface.extractors import NwbRecordingExtractor

client = DandiAPIClient.for_dandi_instance("dandi")

dandiset_id = "000409"
dandiset = client.get_dandiset(dandiset_id)


#asset_path = dandiset_paths_with_ecephys[3]
asset_path = "sub-KS042/sub-KS042_ses-8c552ddc-813e-4035-81cc-3971b57efe65_behavior+ecephys+image.nwb"
recording_asset = dandiset.get_asset_by_path(path=asset_path)
url = recording_asset.get_content_url(follow_redirects=True, strip_query=True)
file_path = url

electrical_series_path = "acquisition/ElectricalSeriesAp00"
recording = NwbRecordingExtractor(file_path=file_path, stream_mode="remfile", electrical_series_path=electrical_series_path)  

In [ ]:
%matplotlib widget

from spikeinterface.widgets import plot_traces


plot_traces(recording=recording, backend="ipywidgets")

In [ ]:
from spikeinterface.widgets import plot_rasters


plot_rasters(sorting=sorting, backend="matplotlib")
